In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

In [2]:
df = pd.read_csv('C:/Users/JenMing/Desktop/MBTI/LSTM/mbti_to_LSTM_DF.csv')
df.head()

,type,posts
0,INFJ,"['INFP', 'INFP', 'INFJ', 'ENFP', 'ISTP', 'INTP..."
1,ENTP,"['INTJ', 'INTP', 'ENFP', 'INTJ', 'INTP', 'INTP..."
2,INTP,"['INTJ', 'INFP', 'INFP', 'INTP', 'INTP', 'INTJ..."
3,INTJ,"['INTJ', 'ISFJ', 'INFP', 'INTP', 'INTP', 'INTP..."
4,ENTJ,"['ENTJ', 'INTP', 'ENFP', 'INTP', 'ENTJ', 'INTJ..."


In [3]:
# 編碼轉換
personality_mapping = {'INFJ': 0,
                        'ENTP': 1,
                        'INTP': 2,
                        'INTJ': 3,
                        'ENTJ': 4,
                        'ENFJ': 5,
                        'INFP': 6,
                        'ENFP': 7,
                        'ISFP': 8,
                        'ISTP': 9,
                        'ISFJ': 10,
                        'ISTJ': 11,
                        'ESTP': 12,
                        'ESFP': 13,
                        'ESTJ': 14,
                        'ESFJ': 15 }

# 資料載入和轉換
encoded_data = []

chars_to_remove = "][' "    

for index, row in df.iterrows():
    dialogues = row["posts"] #字串
    target_personality = row["type"]
    for char in chars_to_remove:
        dialogues = dialogues.replace(char, "")
    
    dialogues_list = dialogues.split(',')
    
    
    dialogue_ids = [personality_mapping[personality] for personality in dialogues_list]
    target_personality_id = personality_mapping[target_personality]
    
    encoded_data.append((dialogue_ids, target_personality_id))
    

In [4]:
# 資料載入和轉換
encoded_data = []

chars_to_remove = "][' "    

for index, row in df.iterrows():
    mbti_counts = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0}
    mbti_per_count = []
    dialogues = row["posts"] #字串
    target_personality = row["type"]
    for char in chars_to_remove:
        dialogues = dialogues.replace(char, "")
    
    dialogues_list = dialogues.split(',')
    
    
    dialogue_ids = [personality_mapping[personality] for personality in dialogues_list]
    
    for personality_id in dialogue_ids:
        mbti_counts[personality_id] += 1
    
    for i in range(len(personality_mapping)):
        mbti_per_count.append(round(mbti_counts[i]/len(dialogue_ids), 2))
    
    target_personality_id = personality_mapping[target_personality]
    
    encoded_data.append((mbti_per_count, target_personality_id))

In [5]:
# 動態計算 input_size
max_dialogue_length = max(len(dialogue) for dialogue, _ in encoded_data)
input_size = max_dialogue_length

In [6]:
# 填充序列並轉換為張量
padded_dialogues = [torch.tensor(dialogue, dtype=torch.float32) for dialogue, _ in encoded_data]
padded_dialogues = pad_sequence(padded_dialogues, batch_first=True)

target_personality = torch.tensor([target for _, target in encoded_data], dtype=torch.float32)

In [7]:
# 检查结果的形状
print(padded_dialogues.shape)
print(target_personality.shape)

torch.Size([8674, 16])
torch.Size([8674])


In [8]:
# 定义 ANN 模型
class PersonalityPredictionANN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_prob=0.5):
        super(PersonalityPredictionANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x

In [9]:
# 資料集切分為訓練集和驗證集
train_dialogues, val_dialogues, train_target, val_target = train_test_split(padded_dialogues, target_personality, test_size=0.15, random_state=42)

# 初始化模型
hidden_size = 64
output_size = len(personality_mapping)
model = PersonalityPredictionANN(input_size, hidden_size, hidden_size, output_size)

In [10]:
# 定義損失函數和優化器 (CEL:分類問題 MSE:回归问题)
criterion = nn.CrossEntropyLoss()
#criterion = nn.MSELoss() 
weight_decay = 0.01
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=weight_decay)
#optimizer = optim.Adam(model.parameters(), lr=0.001)
#StepLR 调度器会每隔 step_size 个周期将学习率乘以 gamma，以逐步降低学习率
#scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

In [11]:
#早停
patience = 10  # 設定早期停止的耐心值
best_val_loss = float('inf')
counter = 0  # 用於計算連續的驗證損失沒有改善的次數

In [12]:
with open("C:/Users/JenMing/Desktop/MBTI/ANN/Model/note.txt", "w") as f:
    num_epochs = 60
    dimension_counts = {'E/I': 0,
                        'S/N': 0,
                        'T/F': 0,
                        'J/P': 0}
    item_count = 0
    for epoch in range(num_epochs):
        model.train()  # 將模型設置為訓練模式
        total_loss = 0.0

        for dialogue_batch, target_batch in zip(train_dialogues, train_target):
            optimizer.zero_grad()

            outputs = model(dialogue_batch)
            loss = criterion(outputs, target_batch.long())  # 使用交叉熵損失
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()

        average_loss = total_loss / len(train_dialogues)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")
        f.write(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}\n")
        
        # 驗證模型
        model.eval()  # 將模型設置為評估模式
        correct_predictions = 0
        total_samples = len(val_dialogues)
        val_loss = 0.0

        with torch.no_grad():
            for dialogue_batch, target_batch in zip(val_dialogues, val_target):
                outputs = model(dialogue_batch)
                loss = criterion(outputs, target_batch.long())  # 使用交叉熵損失
                val_loss += loss.item()

                predicted_class = torch.argmax(outputs).item()
                true_class = target_batch.item()

                for personality, value in personality_mapping.items():
                    if value == predicted_class:
                        mbti_labels_pre = personality
                        break
                for personality, value in personality_mapping.items():
                    if value == int(true_class):
                        mbti_labels_tru = personality 
                        break

                for n in range(4):
                    if n == 0:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['E/I'] += 1
                    elif n == 1:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['S/N'] += 1
                    elif n == 2:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['T/F'] += 1
                    elif n == 3:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['J/P'] += 1

                if predicted_class == true_class:
                    correct_predictions += 1

        item_count += total_samples
        average_val_loss = val_loss / len(val_dialogues)
        accuracy = correct_predictions / total_samples
        print(f"Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {accuracy*100:.4f}%")
        f.write(f"Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {accuracy*100:.4f}%\n")
        
        # 檢查驗證損失是否改善
        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss
            counter = 0
        else:
            counter += 1

        # 如果連續一定次數（耐心值）驗證損失沒有改善，則停止訓練
        if counter >= patience:
            print(f"Early Stopping: Validation loss has not improved for {patience} epochs. Stopping training.")
            break

    EI_counts = dimension_counts['E/I']
    SN_counts = dimension_counts['S/N']
    TF_counts = dimension_counts['T/F']
    JP_counts = dimension_counts['J/P']
    print(f'E.I: {EI_counts}/{item_count} ')
    print('Accuracy: '+ str(EI_counts/item_count)+'\n')
    print(f'S.N: {SN_counts}/{item_count} ')
    print('Accuracy: '+ str(SN_counts/item_count)+'\n')
    print(f'T.F: {TF_counts}/{item_count} ')
    print('Accuracy: '+ str(TF_counts/item_count)+'\n')
    print(f'J.P: {JP_counts}/{item_count} ')
    print('Accuracy: '+ str(JP_counts/item_count)+'\n')
    
    f.write(f'E.I: {EI_counts}/{item_count} ')
    f.write('Accuracy: '+ str(EI_counts/item_count)+'\n')
    f.write(f'S.N: {SN_counts}/{item_count} ')
    f.write('Accuracy: '+ str(SN_counts/item_count)+'\n')
    f.write(f'T.F: {TF_counts}/{item_count} ')
    f.write('Accuracy: '+ str(TF_counts/item_count)+'\n')
    f.write(f'J.P: {JP_counts}/{item_count} ')
    f.write('Accuracy: '+ str(JP_counts/item_count)+'\n')

Epoch [1/60], Loss: 2.3076
Validation Loss: 2.2102, Validation Accuracy: 23.2719%
Epoch [2/60], Loss: 2.2639
Validation Loss: 2.1914, Validation Accuracy: 23.2719%
Epoch [3/60], Loss: 2.2426
Validation Loss: 2.1727, Validation Accuracy: 23.5023%
Epoch [4/60], Loss: 2.2039
Validation Loss: 2.0812, Validation Accuracy: 31.0292%
Epoch [5/60], Loss: 2.0991
Validation Loss: 1.9977, Validation Accuracy: 33.1029%
Epoch [6/60], Loss: 2.0601
Validation Loss: 1.9801, Validation Accuracy: 33.3333%
Epoch [7/60], Loss: 2.0439
Validation Loss: 1.9654, Validation Accuracy: 33.3333%
Epoch [8/60], Loss: 2.0371
Validation Loss: 1.9662, Validation Accuracy: 33.4101%
Epoch [9/60], Loss: 2.0333
Validation Loss: 1.9690, Validation Accuracy: 33.9478%
Epoch [10/60], Loss: 2.0281
Validation Loss: 1.9614, Validation Accuracy: 33.7174%
Epoch [11/60], Loss: 2.0296
Validation Loss: 1.9558, Validation Accuracy: 34.3318%
Epoch [12/60], Loss: 2.0284
Validation Loss: 1.9596, Validation Accuracy: 34.1782%
Epoch [13/60]

In [13]:
torch.save(model.state_dict(), "C:/Users/JenMing/Desktop/MBTI/ANN/Model/best_model.pth")

In [14]:
model.load_state_dict(torch.load('C:/Users/JenMing/Desktop/MBTI/ANN/Model/lr=0.0005 3HL hd_size=64 l2=0.01/best_model.pth'))
model.eval()

PersonalityPredictionANN(
  (fc1): Linear(in_features=16, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (relu2): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=64, out_features=16, bias=True)
)

In [44]:
predictions = []
labels = []
with torch.no_grad():
    for dialogue_batch, target_batch in zip(val_dialogues, val_target):
        outputs = model(dialogue_batch)

        predicted_class = torch.argmax(outputs).item()
        true_class = target_batch.item()
        predictions.append(predicted_class)
        labels.append(true_class)  

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 假設 predictions 和 labels 是你的預測和實際標籤
#cm = confusion_matrix(labels, predictions)
accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')  # 可以使用 'micro'、'macro' 或 'weighted'
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.35023041474654376
Precision: 0.2341618589859333
Recall: 0.35023041474654376
F1 Score: 0.24756675556946836


C:\Users\JenMing\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


改了 正則化的參數 0.001 -> 0.01